In [1]:
# important for gpuhub
# !pip install -r ../../requirements.txt --upgrade

## Importing Libraries and tokens

In [2]:
import os

import wandb
import torch

# load .env file
from dotenv import load_dotenv

from geo_model_tester import GeoModelTester
from image_data_handler_test import TestImageDataHandler
from best_run_loader import BestRunLoader
from wandb_downloader import WandbDownloader

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
WANDB_TOKEN = os.getenv("WANDB_TOKEN")
# Define where to run
env_path = "../../.env"
if not WANDB_TOKEN and os.path.exists(env_path):
    load_dotenv(env_path)
    WANDB_TOKEN = os.getenv("WANDB_TOKEN")

In [4]:
# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available.")

    # Print the name of the GPU
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

    # Print the total and available memory
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert bytes to GB
    print(f"Total Memory: {total_memory:.2f} GB")

    allocated_memory = torch.cuda.memory_allocated(0) / 1e9  # Convert bytes to GB
    print(f"Allocated Memory: {allocated_memory:.2f} GB")

    cached_memory = torch.cuda.memory_reserved(0) / 1e9  # Convert bytes to GB
    print(f"Cached Memory: {cached_memory:.2f} GB")

    # Print other properties
    device_properties = torch.cuda.get_device_properties(0)
    print(f"CUDA Capability: {device_properties.major}.{device_properties.minor}")
    print(f"Multi-Processor Count: {device_properties.multi_processor_count}")
else:
    print("No GPU found.")

No GPU found.


## Loading files from wandb

In [5]:
wandb.login(key=WANDB_TOKEN) if WANDB_TOKEN else wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luki-st (nlp_ls). Use `wandb login --relogin` to force relogin


True

In [6]:
entity = "nlp_ls"
predict_coordinates = False
predict_regions = True
project = "dspro2-predicting-region" if predict_regions else ("dspro2-predicting-coordinates" if predict_coordinates else "dspro2-predicting-country")
metric_name = "Best Validation Accuracy Top 1" if not predict_coordinates else "Best Validation Distance (km)"
metric_ascending = False if not predict_coordinates else True
data_augmentation = "base_augmentation"  # or "full_augmentation_v2"
datasize = 332786  # Replace with the desired datasize
file_names_to_download = [".pth", ".json"]
image_size = [80, 130]

downloader = WandbDownloader(entity, project, data_augmentation, datasize, image_size)
run_data = downloader.get_and_collect_best_runs(metric_name, file_names_to_download, metric_ascending=metric_ascending)

dspro2-predicting-region: Found 5 matching runs for datasize 332786 and base_augmentation.


In [7]:
# Print the validation accuracy for the top 1, 3, and 5 predictions
for j in range(1, min(len(run_data), 6)):
    print(f"\nBest Run {j}: {run_data[f'Best Run {j}']['id']}")
    if predict_coordinates:
        print(f"Best Validation Distance (km): ", run_data[f"Best Run {j}"]["metrics"]["Best Validation Distance (km)"])
    else:
        for i in [1, 3, 5]:
            try:
                print(f"Best Validation Accuracy Top {i}: ", run_data[f"Best Run {j}"]["metrics"][f"Best Validation Accuracy Top {i}"])
            except KeyError:
                print("No validation accuracy found.")


Best Run 1: kwlp08u6
Best Validation Accuracy Top 1:  0.2030440073921601
Best Validation Accuracy Top 3:  0.3429992337395015
Best Validation Accuracy Top 5:  0.4202563216491128

Best Run 2: j3e89vyc
Best Validation Accuracy Top 1:  0.1909340865724116
Best Validation Accuracy Top 3:  0.32034196252835917
Best Validation Accuracy Top 5:  0.3907177306669471

Best Run 3: gtp7iunj
Best Validation Accuracy Top 1:  0.1599380981714921
Best Validation Accuracy Top 3:  0.2780023138062112
Best Validation Accuracy Top 5:  0.34879877398320236

Best Run 4: 9wly2kt2
Best Validation Accuracy Top 1:  0.15693315504004088
Best Validation Accuracy Top 3:  0.27634959508391305
Best Validation Accuracy Top 5:  0.3485734032483435


In [8]:
run_data["Best Run 1"]["parameters"]

{'seed': 42,
 'epochs': 50,
 'optimizer': 'adamW',
 'model_name': 'efficientnet_b1',
 'mapped_data': False,
 'dataset_size': 332786,
 'weight_decay': 0.01,
 'learning_rate': 0.01,
 'predict_regions': True,
 'input_image_size': [80, 130],
 'data_augmentation': 'base_augmentation',
 'different_regions': 4596,
 'dataset_identifier': '22a493044dbe99c1d431b9ee4656792efbb09ece4182274670ba5faec505d9cf',
 'different_countries': 138,
 'predict_coordinates': False}

In [9]:
run_data["Best Run 1"]["files"]

{'wandb_manifest.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/artifact/918441477/wandb_manifest.json',
 'best_model': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/best_model_checkpointmodel_efficientnet_b1_lr_0.01_opt_adamW_weightDecay_0.01_imgSize_[80, 130]_predict_coordinates_False.pth',
 'country_to_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/run-20240623_193550-kwlp08u6/country_to_index.json',
 'region_index_to_country_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/run-20240623_193550-kwlp08u6/region_index_to_country_index.json',
 'region_index_to_middle_point.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/run-20240623_193550-kwlp08u6/region_index_to_middle_point.json',
 'region_to_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/run-20240623_193550-kwlp08u6/region_to_index.json',
 'wandb-metada

## Loading metrics for best runs for each datasize, with and without augmentation

In [10]:
# Example usage:
metric_name = "Best Validation Accuracy Top 1"
project_names = ["coordinates", "region", "country"]
file_names_to_download = [".pth", ".json"]

loader = BestRunLoader(entity, metric_name, project_names, file_names_to_download)

# See the keys of the results
print("\n", loader.results.keys(), "\n")

# See the keys of the results
print(loader.results["dspro2-predicting-region_base_augmentation_81505_(80, 130)"].keys())

dspro2-predicting-coordinates: Found 5 matching runs for datasize 81505 and base_augmentation.
dspro2-predicting-coordinates: Found 5 matching runs for datasize 81505 and full_augmentation_v2.
dspro2-predicting-coordinates: Found 5 matching runs for datasize 332786 and base_augmentation.
Could not find test_data.pth for run wuz2hssk, looks like run fjtmr070 was deleted or is part of a different project.
dspro2-predicting-coordinates: Found 5 matching runs for datasize 332786 and full_augmentation_v2.
dspro2-predicting-coordinates: Found 5 matching runs for datasize 79000 and base_augmentation.
dspro2-predicting-region: Found 5 matching runs for datasize 81505 and base_augmentation.
Run 9e7305d3 is still running, skipping
dspro2-predicting-region: Found 5 matching runs for datasize 81505 and full_augmentation_v2.
dspro2-predicting-region: Found 5 matching runs for datasize 332786 and base_augmentation.
dspro2-predicting-region: Found 5 matching runs for datasize 332786 and full_augmenta

In [11]:
# Count runs per project
project_run_counts = loader.count_runs_per_project()
print(f"Found runs for each project", project_run_counts)

Found runs for each project {'dspro2-predicting-coordinates': 25, 'dspro2-predicting-region': 25, 'dspro2-predicting-country': 21}


In [12]:
# Print the validation accuracy for the top 1, 3, and 5 predictions
coordinates_print = False

for j in range(1, min(len(loader.results["dspro2-predicting-coordinates_base_augmentation_332786_(80, 130)"]), 6)):
    print(f"\nBest Run {j}:", loader.results["dspro2-predicting-country_full_augmentation_v2_332786_(80, 130)"][f"Best Run {j}"][f"id"])
    if coordinates_print:
        print(loader.results["dspro2-predicting-coordinates_base_augmentation_332786_(80, 130)"][f"Best Run {j}"]["metrics"][f"Validation Accuracy Top {i}"])
    else:
        for i in [1, 3, 5]:
            try:
                # print(loader.results["dspro2-predicting-country_base_augmentation_332786_(80, 130)"][f"Best Run {j}"]["metrics"][f"Validation Accuracy Top {i}"])
                print(loader.results["dspro2-predicting-country_full_augmentation_v2_332786_(80, 130)"][f"Best Run {j}"]["metrics"][f"Validation Accuracy Top {i}"])
            except:
                print("No run found")


Best Run 1: bc69qzqh
0.6398575656955692
0.8168336914223898
0.8735670177441892

Best Run 2: kth52fnv
0.6363117328004567
0.8167735925597608
0.8746187478401971

Best Run 3: 8pqe6jmh
0.6054659915561098
0.803356521477831
0.8711780879546854

Best Run 4: elpusqol
0.6046546569106179
0.8028757305767988
0.8695704433793591


## Loading data and creating data loader

In [13]:
cache = True

run = None

for i in range(min(len(run_data), 5)):
    run = run_data[f"Best Run {i+1}"]
    if run["files"].get("test_data", None) and run["files"].get("best_model", None):
        break
    else:
        run = None
        print(f"Run {i+1} does not contain the necessary files. Trying the next run...")

if run is None:
    raise Exception("No run with the necessary files found.")

# Creating Dataloaders with the classes
test_dataset = run["files"]["test_data"]
files = run["files"]
country_to_index = files.get("country_to_index.json", None)
region_to_index = files.get("region_to_index.json", None)
region_index_to_middle_point = files.get("region_index_to_middle_point.json", None)
region_index_to_country_index = files.get("region_index_to_country_index.json", None)

data_handler = TestImageDataHandler(test_dataset, country_to_index, region_to_index, region_index_to_middle_point, region_index_to_country_index, cache=cache)
test_dataloader = data_handler.test_loader
country_to_index = data_handler.country_to_index
region_to_index = data_handler.region_to_index
region_index_to_middle_point = data_handler.region_index_to_middle_point
region_index_to_country_index = data_handler.region_index_to_country_index

num_regions = data_handler.num_regions
num_countries = data_handler.num_countries

Loaded 138 countries.
Loaded 4596 regions.
Loaded 4596 region middle points.
Loaded 3595 region to country index mappings.
Loading test data from test_data.pth
Test data loaded from cache.


## Evaluating the model

In [16]:
num_classes = 3 if predict_coordinates else (num_regions if predict_regions else num_countries)

if num_classes == 0:
    raise ValueError("No classes detected. Please check the data.")

geo_model_tester = GeoModelTester(test_dataloader=test_dataloader, num_classes=num_classes, predict_coordinates=predict_coordinates, country_to_index=country_to_index, region_to_index=region_to_index, region_index_to_middle_point=region_index_to_middle_point, region_index_to_country_index=region_index_to_country_index, predict_regions=predict_regions if not predict_coordinates else None)

/Users/lukasstoeckli/GitLabProjects/dspro2/dspro2/5_evaluation/../4_modeling/geo_model_harness.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  self.region_middle_points = torch.tensor([coordinates_to_cartesian(*coordinate) for coordinate in self.region_middle_points], dtype=torch.float32).to(self.device)


In [17]:
# TODO: Test the model from best runs
# TODO: Show the different models with the best results (also do it for different data sizes and mapped/non-mapped data)
model_name = run["parameters"]["model_name"]
pretrained_weights = run["files"]["best_model"]

# Countries from 81k more mapped dataset, keep in sync with evaluating_models.ipynb
countries_only = ["Albania", "Argentina", "Australia", "Austria", "Bangladesh", "Belgium", "Bolivia, Plurinational State of", "Botswana", "Brazil", "Bulgaria", "Cambodia", "Canada", "Chile", "Colombia", "Croatia", "Czechia", "Denmark", "Dominican Republic", "Ecuador", "Estonia", "Eswatini", "Finland", "France", "Germany", "Ghana", "Greece", "Guatemala", "Hungary", "India", "Indonesia", "Ireland", "Israel", "Italy", "Japan", "Kenya", "Korea, Republic of", "Kyrgyzstan", "Lao People's Democratic Republic", "Latvia", "Lesotho", "Lithuania", "Malaysia", "Malta", "Mexico", "Montenegro", "Netherlands", "New Zealand", "Nigeria", "North Macedonia", "Norway", "Peru", "Philippines", "Poland", "Portugal", "Romania", "Russian Federation", "Rwanda", "Senegal", "Serbia", "Singapore", "Slovakia", "Slovenia", "South Africa", "Spain", "Sri Lanka", "Sweden", "Switzerland", "Thailand", "T\u00fcrkiye", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom", "United States", "Uruguay"]

geo_model_tester.test(model_type=model_name, model_path=pretrained_weights, balanced_on_countries_only=countries_only, accuracy_per_country=False)

if not predict_coordinates:
    # And over all countries
    geo_model_tester.test(model_type=model_name, model_path=pretrained_weights, balanced_on_countries_only=None, accuracy_per_country=True)

/Users/lukasstoeckli/GitLabProjects/dspro2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/lukasstoeckli/GitLabProjects/dspro2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Network-Model: efficientnet_b1
Project Name: region
Run ID: kwlp08u6
Test Loss: 43.8764, Test Distance: 1753.0771, Test Distance Median: 385.5583
Test Top 1 Accuracy: 0.2005, Test Top 3 Accuracy: 0.3405, Test Top 5 Accuracy: 0.4153
Test Top 1 Accuracy (Country): 0.6473, Test Top 3 Accuracy (Country): 0.7085, Test Top 5 Accuracy (Country): 0.7346
Test Top 1 Balanced Accuracy: 0.0845, Test Top 1 Balanced Accuracy (Country): 0.5469
Network-Model: efficientnet_b1
Project Name: region
Run ID: kwlp08u6
Test Loss: 43.8764, Test Distance: 1753.0771, Test Distance Median: 385.5583
Test Top 1 Accuracy: 0.2005, Test Top 3 Accuracy: 0.3405, Test Top 5 Accuracy: 0.4153
Test Top 1 Accuracy (Country): 0.6473, Test Top 3 Accuracy (Country): 0.7085, Test Top 5 Accuracy (Country): 0.7346
Test Top 1 Balanced Accuracy: 0.0845, Test Top 1 Balanced Accuracy (Country): 0.4161
Accuracy per country:
Country Faroe Islands: 1.00000
Country Qatar: 1.00000
Country Rwanda: 0.94737
Country India: 0.93060
Country Jap

/Users/lukasstoeckli/GitLabProjects/dspro2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/lukasstoeckli/GitLabProjects/dspro2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
